In [1]:
import pandas as pd
import folium
from folium import plugins

df1 = pd.read_csv('MDCOVID19_MASTER_ZIP_CODE_CASES.csv')
df2 = pd.read_csv('Vacant_Buildings.csv')

#filter COVID by bmore zips
zipsList = [21227,21207,21230,21251,21229,21237,21287,21231,21226,21206,21222,21225,21211,21208,\
            21205,21218,21234,21202,21201,21213,21210,21209,21216,21236,21217,21224,21215,21212,\
                21223,21214,21239,21228]

df1 = df1[df1['ZIP_CODE'].isin(zipsList)]

df1 = df1.fillna(0)

zipsLatest = pd.DataFrame()

zipsLatest['zipcode1'] = df1['ZIP_CODE'].astype(str)
zipsLatest['LatestCases'] = df1.iloc[:,-1]
zipsLatest = zipsLatest.reset_index()

print(zipsLatest.sort_values(by='LatestCases', ascending=False))

    index zipcode1  LatestCases
18    299    21224       2390.0
12    290    21215       2049.0
3     281    21206       1812.0
16    297    21222       1810.0
26    308    21234       1810.0
4     282    21207       1584.0
23    304    21229       1493.0
22    303    21228       1451.0
5     283    21208       1288.0
15    293    21218       1164.0
28    311    21237       1083.0
24    305    21230       1072.0
21    302    21227       1016.0
10    288    21213        974.0
29    312    21239        970.0
19    300    21225        934.0
27    310    21236        900.0
13    291    21216        810.0
14    292    21217        802.0
1     277    21202        757.0
9     287    21212        744.0
17    298    21223        715.0
6     284    21209        699.0
2     280    21205        617.0
11    289    21214        516.0
25    306    21231        441.0
0     276    21201        395.0
8     286    21211        232.0
20    301    21226        142.0
7     285    21210         96.0
30    31

In [2]:
#filter vacant housing coords
latlong= df2['Location'].astype(str)
latlong = latlong.map(lambda x: x.lstrip('(').rstrip(')'))
latlong = latlong.str.split(expand=True)
latlong.iloc[:,0] = latlong.iloc[:,0].map(lambda x: x.rstrip(','))
latlong.iloc[:,:] = latlong.iloc[:,:].astype(float)
latlong.columns = ['latitude','longitude']
print(latlong.head())

    latitude  longitude
0  39.276713 -76.663622
1  39.290022 -76.636461
2  39.314511 -76.649969
3  39.325083 -76.601653
4  39.358157 -76.581505


In [3]:
#build map
baltMap = folium.Map(location=[39.2904,-76.6122], tiles='OpenStreetMap', zoom_start=13)

folium.GeoJson('zipcode.geojson').add_to(baltMap)

choropleth = folium.Choropleth(geo_data='zipcode.geojson', geo_path='zipcode.geojson',\
                   data=zipsLatest, columns=['zipcode1','LatestCases'],\
                       key_on='feature.properties.zipcode1',fill_color='YlGn',fill_opacity=0.8,\
                           legend_name='COVID-19 Total Reported Cases').add_to(baltMap)

for i,row in latlong.iterrows():
    folium.CircleMarker((row.latitude,row.longitude), radius=3, weight=2, color='red', fill_color='red', fill_opacity=.5).add_to(baltMap)

baltMap.add_child(plugins.HeatMap(data=latlong[['latitude', 'longitude']].to_numpy(), radius=25, blur=10))

choropleth.geojson.add_child(folium.features.GeoJsonTooltip(['zipcode1','zipname'],labels=False))

folium.LayerControl().add_to(baltMap)

baltMap.save('covidByZipMap.html')